In [1]:
pip install azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import AmlCompute
from azure.identity import DefaultAzureCredential

In [3]:
def get_comput_target(ml_client, name="cpu-cluster", family="Standard_D4s_v3"):
    cpu_comput_target = name

    try:
        cpu_cluster = ml_client.compute.get(cpu_comput_target)
    except Exception:
        cpu_cluster = AmlCompute(
            name=cpu_comput_target,
            type="amlcompute",
            size=family,
            min_instances=0,
            max_instances=4,
            idle_time_before_scale_down=180,
            tier="Dedicated"
        )
        cpu_cluster = ml_client.compute.being_create_or_update(cpu_cluster).result()

In [4]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [5]:
compute_target = get_comput_target(ml_client)

In [6]:
clean_data_component = load_component(source="./clean-data/clean.yml")
split_data_component = load_component(source="./split_data/split_data.yml")
save_correlation_component = load_component(source="./save-correlation-img/save_correlation.yml")
train_tree_component = load_component(source="./train_decission_tree/train_decission_tree.yml")
score_component = load_component(source="./score-component/score.yml")
eval_component = load_component(source="./eval-component/eval.yml")

In [7]:
@pipeline(
 default_compute='cpu-cluster',
 )
def water_potability_decision_tree(pipeline_input_data):
    cleaning_node = clean_data_component(
        standard_data=pipeline_input_data
    )

    save_ptl_img_node = save_correlation_component(
        data_set_cleaned = cleaning_node.outputs.cleaned_data
    )

    split_node = split_data_component(
        data_set_cleaned = cleaning_node.outputs.cleaned_data
    )

    train_node = train_tree_component(
        x_train=split_node.outputs.x_train,
        y_train=split_node.outputs.y_train
    )

    score_node = score_component(
        model_input=train_node.outputs.model_output,
        x_test=split_node.outputs.x_test
    )

    eval_node = eval_component(
        scoring_result=score_node.outputs.y_pred,
        y_test=split_node.outputs.y_test
    )

    return {
        "pipeline_ptl_img_output": save_ptl_img_node.outputs.correlation_img,
        "pipeline_model_dtree_output": train_node.outputs.model_output,
        "pipeline_eval_output": eval_node.outputs.eval_output
    }


In [9]:
# create a pipeline
water_potability_proy2_ds = Input(
    type="uri_file",
    path="azureml://subscriptions/43811fcd-1f9c-4778-a8ec-956a4768f3b9/resourcegroups/andrew.j.t.p-rg/workspaces/pipelines/datastores/workspaceblobstore/paths/UI/2023-11-04_191851_UTC/water_potability_ds.csv",
)
pipeline_job = water_potability_decision_tree(pipeline_input_data=water_potability_proy2_ds)


In [10]:
pipeline_job = ml_client.jobs.create_or_update(
pipeline_job, experiment_name="pipeline_water_potability_proy2")
pipeline_job

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading save_correlation_src (0.0 MB

Experiment,Name,Type,Status,Details Page
pipeline_water_potability_proy2,boring_corn_ckz9x4b9cz,pipeline,Preparing,Link to Azure Machine Learning studio


In [11]:
ml_client.jobs.stream(pipeline_job.name)

RunId: boring_corn_ckz9x4b9cz
Web View: https://ml.azure.com/runs/boring_corn_ckz9x4b9cz?wsid=/subscriptions/43811fcd-1f9c-4778-a8ec-956a4768f3b9/resourcegroups/andrew.j.t.p-rg/workspaces/pipelines

Streaming logs/azureml/executionlogs.txt

[2023-11-06 15:56:13Z] Completing processing run id c83c5cb2-80a4-4639-94ec-468a35eb4d45.
[2023-11-06 15:56:15Z] Completing processing run id faa2449f-60f1-4ca9-aef3-43743b5e6682.
[2023-11-06 15:56:16Z] Completing processing run id cedc5364-87c6-4ef0-aa91-b39127aeb9ad.
[2023-11-06 15:56:17Z] Completing processing run id 76a80952-32ad-45e1-a655-310ff5338471.
[2023-11-06 15:56:20Z] Completing processing run id e397ef17-bb4c-47c4-84dd-0f4fb4511430.
[2023-11-06 15:56:21Z] Submitting 1 runs, first five are: ea7f953e:93a3df47-2316-4559-a1f1-3ff02d61eb16
[2023-11-06 16:02:26Z] Completing processing run id 93a3df47-2316-4559-a1f1-3ff02d61eb16.

Execution Summary
RunId: boring_corn_ckz9x4b9cz
Web View: https://ml.azure.com/runs/boring_corn_ckz9x4b9cz?wsid=/s

In [12]:
output = ml_client.jobs.download(name=pipeline_job.name, download_path='./pipeline_output', all=True)